In [1]:
import numpy as np
import pandas as pd

In [2]:
texts_month = pd.read_csv('data/texts_31days.csv', index_col=0)
texts_first = pd.read_csv('data/texts_12.04-13.04.csv', index_col=0)
texts_second = pd.read_csv('data/texts_13.04-14.04.csv', index_col=0)
texts = pd.concat([texts_month, texts_first, texts_second])
texts.index.names = ['url_id']

texts.to_csv('data/texts.csv')

In [14]:
from datetime import datetime
from datetime import timezone
from zipfile import ZipFile
import os

def make_df(start_time, end_time):
    timestamps = sorted(os.listdir('data/timestamps'))
    start_timestamp, end_timestamp = make_timestamps_from_datetime(start_time, end_time, timestamps)
    interval = make_interval(start_timestamp, end_timestamp, timestamps)
    with ZipFile('data/timestamps.zip') as timestamps_zip:
        df_list = [pd.read_csv(timestamps_zip.open("timestamps/" + file), header=None, names=['fullVisitorId', 'url_id', 'visitStartTime']) for file in interval]
    df = pd.concat(df_list)
    labels, levels = pd.factorize(df['fullVisitorId'])
    df['user_id'] = labels
    df.set_index(['user_id','url_id'], inplace=True, drop=True)
    return df

# first = 12/03/2017 07:00:00, last = 14/04/2017 11:11:29 1491818423 1491991225
def make_timestamps_from_datetime(start_time, end_time, timestamps):
    if start_time == 'first':
        start_timestamp = timestamps[0]
    else:
        start_datetime = datetime.strptime(start_time, '%d/%m/%Y %H:%M:%S')
        start_timestamp = (start_datetime - datetime(1970, 1, 1)).total_seconds()
        
    if end_time == 'last':
        end_timestamp = timestamps[-1]
    else:
        end_datetime = datetime.strptime(end_time, '%d/%m/%Y %H:%M:%S')
        end_timestamp = (end_datetime - datetime(1970, 1, 1)).total_seconds()
    return (start_timestamp, end_timestamp)

def make_interval(start_timestamp, end_timestamp, timestamps):
    start_timestamp = str(start_timestamp)
    end_timestamp = str(end_timestamp)
    interval = [t for t in timestamps if t >= start_timestamp and t <= end_timestamp]
    return interval

# Using texts.csv to make urls for each url_id
def make_urls_df():
    texts = pd.read_csv('data/texts.csv')
    tag_cleaned = texts['tag'].str.split().str.get(0)
    texts['tag_cleaned'] = tag_cleaned
    texts['url_id'] = texts['url_id'].astype(str)
    texts['pagePath'] = '/t/' + texts['tag_cleaned'] + '/' + texts['url_id']
    texts.set_index(['url_id'], inplace=True)
    urls = texts.drop(['subtitle', 'tag', 'title', 'tag_cleaned'], axis=1)
    return urls

def merge_df(df, urls):
    df.reset_index(level=['url_id'], inplace=True)
    df.reset_index(level=['user_id'], inplace=True)
    urls.reset_index(level=['url_id'], inplace=True)
    urls['url_id'] = urls['url_id'].astype(int)
    df['fullVisitorId'] = df['fullVisitorId'].astype(str)
    df_result = pd.merge(df, urls, on='url_id', how='left')
    labels, levels = pd.factorize(df_result['url_id'])
    df_result['url_id'] = labels
    df_result.set_index(['user_id', 'url_id'], inplace=True)
    return df_result

df = make_df('15/03/2017 10:00:00', '16/03/2017 10:00:00')
urls = make_urls_df()
df_result = merge_df(df, urls)

In [15]:
display(df_result.info())
display(df_result.head(150))
display(df_result.tail(150))

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11969533 entries, (0, 0) to (1012770, 12641)
Data columns (total 3 columns):
fullVisitorId     object
visitStartTime    int64
pagePath          object
dtypes: int64(1), object(2)
memory usage: 373.4+ MB


None

fullVisitorId  visitStartTime                pagePath
user_id url_id                                                              
0       0        1036536157958312061      1489571954     /t/экономика/939611
        0        1036536157958312061      1489571954     /t/экономика/939611
1       1       -4611953592302795458      1489571994       /t/новости/163224
        1       -4611953592302795458      1489571994       /t/новости/163224
        1       -4611953592302795458      1489571994       /t/новости/163224
2       2        5259739520833585814      1489572000  /t/происшествия/985017
        2        5259739520833585814      1489572000       /t/новости/985017
        2        5259739520833585814      1489572000       /t/новости/985017
3       2        6061855442569952128      1489571984  /t/происшествия/985017
        2        6061855442569952128      1489571984       /t/новости/985017
        2        6061855442569952128      1489571984       /t/новости/985017
4       2        7440905335861206372      1489571952  /t/происшествия/985017
        2        7440905335861206372      1489571952       /t/новости/985017
        2        7440905335861206372      1489571952       /t/новости/985017
5       3       -1862798276330638216      1489571963  /t/происшествия/972874
        3       -1862798276330638216      1489571963       /t/новости/972874
        3       -1862798276330638216      1489571963       /t/новости/972874
6       4        8635801096679781758      1489571958       /t/корабли/985103
7       5       -7713500338258007951      1489571991  /t/происшествия/984980
        5       -7713500338258007951      1489571991       /t/новости/984980
        5       -7713500338258007951      1489571991       /t/новости/984980
8       6       -3478535520227556835      1489571973     /t/димабилан/983987
        6       -3478535520227556835      1489571973     /t/димабилан/983987
        6       -3478535520227556835      1489571973     /t/димабилан/983987
9       7        3426149204546475971      1489571943     /t/гватемала/983399
10      8        -936696997817697392      1489571945      /t/скандалы/949611
11      8        -115472236325987330      1489571979      /t/скандалы/949611
12      8        3294159398504767341      1489571966      /t/скандалы/949611
13      8        4633023777466028437      1489571943      /t/скандалы/949611
14      8       -7954465332243381894      1489571949      /t/скандалы/949611
...                              ...             ...                     ...
48      9        8511667242330154662      1489571952        /t/life78/985495
        9        8511667242330154662      1489571952        /t/life78/985495
49      9       -9155659245588143684      1489571978     /t/школьники/985495
        9       -9155659245588143684      1489571978        /t/life78/985495
        9       -9155659245588143684      1489571978        /t/life78/985495
50      9       -4122865017404534947      1489571948     /t/школьники/985495
        9       -4122865017404534947      1489571948        /t/life78/985495
        9       -4122865017404534947      1489571948        /t/life78/985495
51      9       -6014338222960092841      1489571954     /t/школьники/985495
        9       -6014338222960092841      1489571954        /t/life78/985495
        9       -6014338222960092841      1489571954        /t/life78/985495
52      9       -8041207334040544641      1489571991     /t/школьники/985495
        9       -8041207334040544641      1489571991        /t/life78/985495
        9       -8041207334040544641      1489571991        /t/life78/985495
53      9       -4440654952647091355      1489571987     /t/школьники/985495
        9       -4440654952647091355      1489571987        /t/life78/985495
        9       -4440654952647091355      1489571987        /t/life78/985495
54      9       -6142093398951167810      1489571982     /t/школьники/985495
        9       -6142093398951167810      1489571982        /t/life78/985495
        9       -61

fullVisitorId  visitStartTime              pagePath
user_id url_id                                                            
934528  2347      442690634461732406      1489658313      /t/мнения/984473
        2347      442690634461732406      1489658313      /t/мнения/984473
1002862 13238    3373014268182322385      1489658288  /t/технологии/985616
        13238    3373014268182322385      1489658288        /t/авто/985616
1013315 13238    7198888419577621261      1489658302  /t/технологии/985616
        13238    7198888419577621261      1489658302        /t/авто/985616
294489  40031    7121298180140998711      1489658318       /t/спорт/985929
939873  40031   -1494589981853249330      1489658303       /t/спорт/985929
434859  40031   -4998192895174106326      1489658343       /t/спорт/985929
1008658 40031   -7196422238886125038      1489658305       /t/спорт/985929
1285    40031     612121713321166117      1489658315       /t/спорт/985929
831534  40031    4603394634898891905      1489658325       /t/спорт/985929
554281  40031   -6018684220756614321      1489658342       /t/спорт/985929
1012990 40031    1612349859104039478      1489658335       /t/спорт/985929
1009919 40031   -4663990341278447826      1489658317       /t/спорт/985929
803317  40031    5907718266717505349      1489658289       /t/спорт/985929
982777  40031   -7141776883677330329      1489658306       /t/спорт/985929
929849  40031     636297737926232993      1489658292       /t/спорт/985929
1009801 40031    4671237180739844391      1489658299       /t/спорт/985929
2920    40031   -8915620738510309630      1489658310       /t/спорт/985929
999354  40031   -8334946674570987717      1489658336       /t/спорт/985929
1005356 40031     199499220158840624      1489658335       /t/спорт/985929
1013494 40031   -7814140578579083082      1489658301       /t/спорт/985929
774001  40031    6997013154368127849      1489658303       /t/спорт/985929
1006703 40031   -2623380145707808824      1489658307       /t/спорт/985929
805322  40031    4567290398516492673      1489658342       /t/спорт/985929
826645  40031     541649939939448680      1489658337       /t/спорт/985929
110778  40031    8510985925761635958      1489658311       /t/спорт/985929
850837  40031    4633775000745676686      1489658332       /t/спорт/985929
99165   40031    1728972646627284915      1489658339       /t/спорт/985929
...                              ...             ...                   ...
1014785 43205    8696982156960646426      1489658337     /t/новости/986011
        43205    8696982156960646426      1489658337     /t/новости/986011
1014363 43205    8448751300048861237      1489658318  /t/референдум/986011
        43205    8448751300048861237      1489658318     /t/новости/986011
        43205    8448751300048861237      1489658318     /t/новости/986011
309049  43205     238944299085452104      1489658345  /t/референдум/986011
        43205     238944299085452104      1489658345     /t/новости/986011
        43205     238944299085452104      1489658345     /t/новости/986011
1014977 43205    4754752333300011806      1489658289  /t/референдум/986011
        43205    4754752333300011806      1489658289     /t/новости/986011
        43205    4754752333300011806      1489658289     /t/новости/986011
932216  43205   -1651808196323568760      1489658328  /t/референдум/986011
        43205   -1651808196323568760      1489658328     /t/новости/986011
        43205   -1651808196323568760      1489658328     /t/новости/986011
161009  43205   -8540355032294878802      1489658287  /t/референдум/986011
        43205   -8540355032294878802      1489658287     /t/новости/986011
        43205   -8540355032294878802      1489658287     /t/новости/986011
1014978 40773   -6738271101230342151      1489658288        /t/фото/921137
        40773   -6738271101230342151      1489658288     /t/новости/921137
        40773   -6738271101230342151      1489658288     /t/новости/921137
1000659 800      2600425054464374697      148965